In [12]:
import numpy as np
import csv


In [13]:
H = 7
G = 1.5
F = 1.5
N = 1
L = 2
M = 2
transform_matrix = np.array( [ [ (H+G), -H, -G, 0, 0, 0 ], [ -H, (F+H), -F, 0, 0, 0 ], [ -G, -F, (F+G), 0, 0, 0 ], [ 0, 0, 0, 2*N, 0, 0 ], [ 0, 0, 0, 0, 2*L, 0], [ 0, 0, 0, 0, 0, 2*M] ])

In [14]:
display(transform_matrix)

array([[ 8.5, -7. , -1.5,  0. ,  0. ,  0. ],
       [-7. ,  8.5, -1.5,  0. ,  0. ,  0. ],
       [-1.5, -1.5,  3. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  2. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  4. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  4. ]])

In [15]:
def test_hill_YS(stress_vector):
    s11 = stress_vector[0]
    s22 = stress_vector[1]
    s33 = stress_vector[2]
    s23 = stress_vector[3]
    s13 = stress_vector[4]
    s12 = stress_vector[5]
    hill_yield = np.sqrt(F*(s22 - s33)**2 + G*(s33 - s11)**2 + H*(s11 - s22)**2 + 2*L*s23 + 2*M*s13**2 + 2*N*s12**2)
    return hill_yield

In [4]:
def von_mises_stress(voigt_stress):
    sigma_11 = voigt_stress[0]
    sigma_22 = voigt_stress[1]
    sigma_33 = voigt_stress[2]
    sigma_23 = voigt_stress[3]
    sigma_13 = voigt_stress[4]
    sigma_12 = voigt_stress[5]
    vm_sq = 1/2*( (sigma_11 - sigma_22)**2 + (sigma_22 - sigma_33)**2 + (sigma_33 - sigma_11)**2 + 6*(sigma_23**2 + sigma_13**2 + sigma_12**2) )
    return np.sqrt(vm_sq) * 100.0
axial_and_shear_pairs = []
for uniax in np.linspace(0,1,5):
    equiv_shear = np.sqrt((1. - uniax**2)/3)
    axial_and_shear_pairs.append([uniax, equiv_shear])


In [5]:
s11_12 = [ [s11, 0, 0, s12, 0, 0] for s11, s12 in axial_and_shear_pairs ]
s11_13 = [ [s11, 0, 0, 0, 0, s13] for s11, s13 in axial_and_shear_pairs ]
s11_23 = [ [s11, 0, 0, 0, s23, 0] for s11, s23 in axial_and_shear_pairs ]
s22_12 = [ [0, s22, 0, s12, 0, 0] for s22, s12 in axial_and_shear_pairs ]
s22_13 = [ [0, s22, 0, 0, 0, s13] for s22, s13 in axial_and_shear_pairs ]
s22_23 = [ [0, s22, 0, 0, s23, 0] for s22, s23 in axial_and_shear_pairs ]
s33_12 = [ [0, 0, s33, s12, 0, 0] for s33, s12 in axial_and_shear_pairs ]
s33_13 = [ [0, 0, s33, 0, 0, s13] for s33, s13 in axial_and_shear_pairs ]
s33_23 = [ [0, 0, s33, 0, s23, 0] for s33, s23 in axial_and_shear_pairs ]
all_stress_combos = s11_12 + s11_13 + s11_23 + s22_12 + s22_13 + s22_23 + s33_12 + s33_13 + s33_23
for s in all_stress_combos: 
    vm_stress = von_mises_stress(s)
    assert(vm_stress == 1.0)

In [10]:
def test_hill_YS(stress_vector):
    s11 = stress_vector[0]
    s22 = stress_vector[1]
    s33 = stress_vector[2]
    s23 = stress_vector[3]
    s13 = stress_vector[4]
    s12 = stress_vector[5]
    hill_yield = np.sqrt(F*(s22 - s33)**2 + G*(s33 - s11)**2 + H*(s11 - s22)**2 + 2*L*s23 + 2*M*s13**2 + 2*N*s12**2)
    return hill_yield

In [11]:
stresses = []
for stress in all_stress_combos:
    stress = np.array(stress)
    new_yield_stress = transform_matrix @ stress
    stresses.append(list(new_yield_stress))
stresses = np.array(stresses)

for stress_vec in stresses:
    print(stress_vec, test_hill_YS(stress_vec))


# with open('epvm_eq_0.csv', 'wb') as myfile:
#     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
#     wr.writerow(stresses)

np.savetxt('epvm_eq_0.csv', stresses, delimiter=",")

[0.         0.         0.         1.15470054 0.         0.        ] 2.149139863647084
[ 2.         -1.75       -0.25        1.11803399  0.          0.        ] 10.498673056867691
[ 4.  -3.5 -0.5  1.   0.   0. ] 20.663978319771825
[ 6.         -5.25       -0.75        0.76376262  0.          0.        ] 30.899919910305655
[ 8. -7. -1.  0.  0.  0.] 41.13392760240627
[0.         0.         0.         0.         0.         2.30940108] 3.265986323710904
[ 2.         -1.75       -0.25        0.          0.          2.23606798] 10.758717395675006
[ 4.  -3.5 -0.5  0.   0.   2. ] 20.760539492026695
[ 6.         -5.25       -0.75        0.          0.          1.52752523] 30.92598691499863
[ 8. -7. -1.  0.  0.  0.] 41.13392760240627
[0.         0.         0.         0.         2.30940108 0.        ] 4.618802153517006
[ 2.         -1.75       -0.25        0.          2.23606798  0.        ] 11.213830746002902
[ 4.  -3.5 -0.5  0.   2.   0. ] 20.952326839756964
[ 6.         -5.25       -0.75       

In [ ]:
stresses_evmp = []
for stress in all_stress_combos:
    stress = np.array(stress)
    new_yield_stress = (1.1*transform_matrix) @ stress
    stresses_evmp.append(list(new_yield_stress))
stresses_evmp = np.array(stresses_evmp)
print(stresses_evmp)

np.savetxt('epvm_deq_0.csv', stresses_evmp, delimiter=",")

In [ ]:
stresses_evmp = []
for stress in all_stress_combos:
    stress = np.array(stress)
    new_yield_stress = (1.2*transform_matrix) @ stress
    stresses_evmp.append(list(new_yield_stress))
stresses_evmp = np.array(stresses_evmp)
print(stresses_evmp)

np.savetxt('epvm_deq_1.csv', stresses_evmp, delimiter=",")

In [ ]:
stresses_evmp = []
for stress in all_stress_combos:
    stress = np.array(stress)
    new_yield_stress = (1.3*transform_matrix) @ stress
    stresses_evmp.append(list(new_yield_stress))
stresses_evmp = np.array(stresses_evmp)
print(stresses_evmp)

np.savetxt('epvm_deq_2.csv', stresses_evmp, delimiter=",")

In [ ]:
stresses_evmp = []
for stress in all_stress_combos:
    stress = np.array(stress)
    new_yield_stress = (1.4*transform_matrix) @ stress
    stresses_evmp.append(list(new_yield_stress))
stresses_evmp = np.array(stresses_evmp)
print(stresses_evmp)

np.savetxt('epvm_deq_4.csv', stresses_evmp, delimiter=",")